In [1]:
import os
import json
import torch

from models import get_model, InvariantNet
from data import get_datamodule
from torchsummary import summary
import pytorch_lightning as pl
torch.set_float32_matmul_precision('high')

def load_model(model_name, device):
    model_path = f"{model_name}.ckpt"
    assert os.path.exists(model_path), f"There is no model at {model_path}"
    with open(f"{model_name}.json") as config_file:
        config = json.load(config_file)
    model = torch.load(model_path, map_location=device)
    state_dict = model['state_dict']
    datamodule = get_datamodule(config['datamodule_name'])(**config['datamodule_hparams'])
    hnet_backbone = get_model(model_name=config['backbone_name'],
                              model_hparams=config['backbone_hparams'])
    pred_net = get_model(model_name=config['classnet_name'],
                         model_hparams=config['classnet_hparams'])

    # model
    model = InvariantNet(hnet_backbone,
                         pred_net,
                         input_shape=datamodule.sample_input_shape,
                         log_invariance=False)
    model.load_state_dict(state_dict)
    model.freeze()
    datamodule.setup('')
    return datamodule, model

trainer = pl.Trainer(accelerator="gpu" if torch.cuda.is_available() else "auto",
                     devices=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


# Choose model
* e.g. "./data/models/MNIST/H-Nets" from [data/models](./data/models)

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
artifact_name = "./data/models/rot-MNIST/UP+MASK"
ds, model = load_model(artifact_name, device)

  0%|          | 0/58424278 [00:00<?, ?it/s]

Extracting data/mnist_rotation_new.zip to data


# Model Summary

In [3]:
_ = summary(model, input_data=ds.sample_input_shape[1:])

Layer (type:depth-idx)                   Output Shape              Param #
├─UpscaleHnetBackbone: 1-1               [-1, 16, 16, 10]          --
|    └─Upsample: 2-1                     [-1, 1, 64, 64]           --
|    └─HView: 2-2                        [-1, 64, 64, 1, 1, 1]     --
|    └─Sequential: 2-3                   [-1, 32, 32, 3, 2, 8]     --
|    |    └─UpscaleHConv2d: 3-1          [-1, 64, 64, 3, 2, 8]     (9,394)
|    |    └─HNonlin: 3-2                 [-1, 64, 64, 3, 2, 8]     (24)
|    |    └─UpscaleHConv2d: 3-3          [-1, 64, 64, 3, 2, 8]     (10,066)
|    |    └─HBatchNorm: 3-4              [-1, 64, 64, 3, 2, 8]     (16)
|    |    └─HMeanPool: 3-5               [-1, 32, 32, 3, 2, 8]     --
|    └─Sequential: 2-4                   [-1, 16, 16, 3, 2, 16]    --
|    |    └─UpscaleHConv2d: 3-6          [-1, 32, 32, 3, 2, 16]    (7,762)
|    |    └─HNonlin: 3-7                 [-1, 32, 32, 3, 2, 16]    (48)
|    |    └─UpscaleHConv2d: 3-8          [-1, 32, 32, 3, 2, 16]

# Test Accuracy

In [4]:
_ = trainer.test(model=model,
                 datamodule=ds,
                 verbose=True)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9860000014305115     │
│         test_loss         │    0.1551932543516159     │
└───────────────────────────┴───────────────────────────┘